In [55]:
import pandas as pd
import numpy as np
import scipy.stats
import warnings

warnings.filterwarnings("ignore")
gams_python_path="/Library/Frameworks/GAMS.framework/Resources/apifiles/Python/gams"

#### Import Data

In [56]:
# manual=pd.read_excel('/Users/hannahkamen/Downloads/msa_2019_lookup.xlsx')

# old_msas=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/0610_msa_lookup.dta')
# old_msas=old_msas[old_msas['metarea_07_lookup']!='not identifiable or not in an msa']
# old_msas['metarea_07_lookup']=old_msas['metarea_07_lookup'].str.replace('opekika','opelika')

# cbsa=pd.read_excel('/Users/hannahkamen/Downloads/msa_county_2013.xlsx',converters={'FIPS State Code': str,'FIPS County Code': str,'CSA Code': str})
# cbsa['msa_name']=cbsa['CBSA Title'].str.lower()



In [57]:
# msa_2007=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/msa_2007_lookup.dta')
####### import msa-specific (fixed) amenity data
#population and race 2019
pop_race=pd.read_csv('/Users/hannahkamen/Downloads/CCDB_updated_vars/pop_race2019.csv')
#income
acs=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/acs5yr2019.dta')
####get list of updated msas in new data
new_msas=acs.groupby('met2013',as_index=False).sum()[['met2013']]
#crime
crime=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/crime_fbi_2018.xlsx')
#manufacturing
manu = pd.read_csv("/Users/hannahkamen/Downloads/CCDB_updated_vars/2019.annual.singlefile.csv")
#government spending
fed_spend=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/federal_spending.xlsx')
#old CCDB data
old_ccdb=pd.read_stata('/Users/hannahkamen/Downloads/CCDB_updated_vars/ccdb_2007_modified.dta')



In [58]:
####get list of updated msas in new data
# new_msas=acs.groupby('met2013',as_index=False).sum()[['met2013']]
# new_msas=new_msas[new_msas['met2013']!='Not in identifiable area']
# new_msas=new_msas.rename(columns={'met2013':'metarea_19'})

#### Crime Data

In [59]:
#import crime data
crime1=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/crime_fbi_2019.xlsx')
crime2=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/crime_fbi_2018.xlsx')
crime3=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/crime_fbi_2017.xls')
crime4=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/crime_fbi_2015.xls')


crime1=crime1[crime1['Metropolitan Statistical Area']!='Washington-Arlington-Alexandria, DC-VA-MD-WV M.D.3, 4']
crime2=crime2[crime2['Metropolitan Statistical Area']!='Washington-Arlington-Alexandria, DC-VA-MD-WV M.D.']
crime3=crime3[crime3['Metropolitan Statistical Area']!='Washington-Arlington-Alexandria, DC-VA-MD-WV M.D.']
crime4=crime4[crime4['Metropolitan Statistical Area']!='Washington-Arlington-Alexandria, DC-VA-MD-WV M.D.']


###clean msq variable
crime1['msa_clean']=crime1['Metropolitan Statistical Area'].str.replace('\d+', '').str.lower().str.replace(' m.s.a.','').str.replace(' m.d.','').str.replace('M.S.A.','').str.replace(' m.s.a','').str.replace(', , ,','').str.strip().str.rstrip(',')
crime2['msa_clean']=crime2['Metropolitan Statistical Area'].str.replace('\d+', '').str.lower().str.replace(' m.s.a.','').str.replace(' m.d.','').str.replace('M.S.A.','').str.replace(' m.s.a','').str.replace(', , ,','').str.strip().str.rstrip(',')
crime3['msa_clean']=crime3['Metropolitan Statistical Area'].str.replace('\d+', '').str.lower().str.replace(' m.s.a.','').str.replace(' m.d.','').str.replace('M.S.A.','').str.replace(' m.s.a','').str.replace(', , ,','').str.strip().str.rstrip(',')
crime4['msa_clean']=crime4['Metropolitan Statistical Area'].str.replace('\d+', '').str.lower().str.replace(' m.s.a.','').str.replace(' m.d.','').str.replace('M.S.A.','').str.replace(' m.s.a','').str.replace(', , ,','').str.strip().str.rstrip(',')



####get population
crime_pop=crime1[~(crime1['msa_clean'].isnull())]
crime_pop=crime_pop[['Population','msa_clean']]
####get number of crimes
crime1['msa_clean']=crime1['msa_clean'].fillna(method='ffill')
crime_count=crime1[(crime1['Counties/principal cities'].astype(str).str.contains('actually'))]
crime_count['total_crime_count']=crime_count['Murder and\nnonnegligent\nmanslaughter'].fillna(0)+crime_count['Rape1'].fillna(0)+crime_count['Robbery'].fillna(0)+crime_count['Aggravated\nassault'].fillna(0)+crime_count['Burglary'].fillna(0)+crime_count['Larceny-\ntheft'].fillna(0)+crime_count['Motor\nvehicle\ntheft'].fillna(0)
crime_count=crime_count[['msa_clean','total_crime_count']]
####merge
crime_m1=crime_count.merge(crime_pop,how='inner')
crime_m1['crime_per_cap']=crime_m1['total_crime_count']/crime_m1['Population']


####get population
crime_pop=crime2[~(crime2['msa_clean'].isnull())]
crime_pop=crime_pop[['Population','msa_clean']]
####get number of crimes
crime2['msa_clean']=crime2['msa_clean'].fillna(method='ffill')
crime_count=crime2[(crime2['Counties/principal cities'].astype(str).str.contains('actually'))]
crime_count['total_crime_count']=crime_count['Murder and\nnonnegligent\nmanslaughter'].fillna(0)+crime_count['Rape1'].fillna(0)+crime_count['Robbery'].fillna(0)+crime_count['Aggravated\nassault'].fillna(0)+crime_count['Burglary'].fillna(0)+crime_count['Larceny-\ntheft'].fillna(0)+crime_count['Motor\nvehicle\ntheft'].fillna(0)
crime_count=crime_count[['msa_clean','total_crime_count']]
####merge
crime_m2=crime_count.merge(crime_pop,how='inner')
crime_m2['crime_per_cap']=crime_m2['total_crime_count']/crime_m2['Population']


####get population
crime_pop=crime3[~(crime3['msa_clean'].isnull())]
crime_pop=crime_pop[['Population','msa_clean']]
####get number of crimes
crime3['msa_clean']=crime3['msa_clean'].fillna(method='ffill')
crime_count=crime3[(crime3['Counties/principal cities'].astype(str).str.contains('actually'))]
crime_count['total_crime_count']=crime_count['Murder and\nnonnegligent\nmanslaughter'].fillna(0)+crime_count['Rape1'].fillna(0)+crime_count['Robbery'].fillna(0)+crime_count['Aggravated\nassault'].fillna(0)+crime_count['Burglary'].fillna(0)+crime_count['Larceny-\ntheft'].fillna(0)+crime_count['Motor\nvehicle\ntheft'].fillna(0)
crime_count=crime_count[['msa_clean','total_crime_count']]
####merge
crime_m3=crime_count.merge(crime_pop,how='inner')
crime_m3['crime_per_cap']=crime_m3['total_crime_count']/crime_m3['Population']


####get population
crime_pop=crime4[~(crime4['msa_clean'].isnull())]
crime_pop=crime_pop[['Population','msa_clean']]
####get number of crimes
crime4['msa_clean']=crime4['msa_clean'].fillna(method='ffill')
crime_count=crime4[(crime4['Counties/principal cities'].astype(str).str.contains('actually'))]
crime_count['total_crime_count']=crime_count['Murder and\nnonnegligent\nmanslaughter'].fillna(0)+crime_count['Rape1'].fillna(0)+crime_count['Robbery'].fillna(0)+crime_count['Aggravated\nassault'].fillna(0)+crime_count['Burglary'].fillna(0)+crime_count['Larceny-\ntheft'].fillna(0)+crime_count['Motor\nvehicle\ntheft'].fillna(0)
crime_count=crime_count[['msa_clean','total_crime_count']]
####merge
crime_m4=crime_count.merge(crime_pop,how='inner')
crime_m4['crime_per_cap']=crime_m4['total_crime_count']/crime_m3['Population']


####append if not in 2019
from_2018=[x for x in crime_m2['msa_clean'].unique() if x not in crime_m1['msa_clean'].unique()]
crime_m2=crime_m2[crime_m2['msa_clean'].isin(from_2018)]

####append
crime_m=crime_m1.append(crime_m2)

from_2017=[x for x in crime_m3['msa_clean'].unique() if x not in crime_m['msa_clean'].unique()]
crime_m3=crime_m3[crime_m3['msa_clean'].isin(from_2017)]

crime_m=crime_m.append(crime_m3).reset_index()


from_2015=[x for x in crime_m4['msa_clean'].unique() if x not in crime_m['msa_clean'].unique()]
crime_m4=crime_m4[crime_m4['msa_clean'].isin(from_2015)]

crime_m=crime_m.append(crime_m4).reset_index()


#### Manufacturing Data

In [60]:
manu['industry_code_lm']=manu['industry_code'].astype(str).apply(lambda x: x[0:2])
manu_lm=manu[~(manu['area_fips'].astype(str).str.contains("[a-zA-Z]")) & (manu['industry_code_lm'].isin(['31','32','33']))]
manu_lm['area_fips']=manu_lm['area_fips'].astype(int).astype(str)


In [61]:
###import fips
fips_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msa_county_2019 (1).xlsx')
fips_lookup=fips_lookup[~(fips_lookup['FIPS State Code'].isnull())]

fips_lookup['len_state']=fips_lookup['FIPS State Code'].astype(int).astype(str).apply(lambda x: len(x))
fips_lookup['len_county']=fips_lookup['FIPS County Code'].astype(int).astype(str).apply(lambda x: len(x))

fips_lookup['clean_state']=fips_lookup['FIPS State Code'].astype(int).astype(str)
fips_lookup['clean_county']=fips_lookup['FIPS County Code'].astype(int).astype(str)

fips_lookup['fips_clean']=np.where((fips_lookup['len_state']==1)& (fips_lookup['len_county']==2),fips_lookup['clean_state']+"0"+fips_lookup['clean_county'],'')
fips_lookup['fips_clean']=np.where((fips_lookup['len_state']==1)& (fips_lookup['len_county']==3),fips_lookup['clean_state']+fips_lookup['clean_county'],fips_lookup['fips_clean'])

fips_lookup['fips_clean']=np.where((fips_lookup['len_state']==2) & (fips_lookup['len_county']<=2),fips_lookup['clean_state']+"0"+fips_lookup['clean_county'],fips_lookup['fips_clean'])
fips_lookup['fips_clean']=np.where((fips_lookup['len_state']==2) & (fips_lookup['len_county']>=3),fips_lookup['clean_state']+fips_lookup['clean_county'],fips_lookup['fips_clean'])
 
fips_lookup['fips_clean']=np.where((fips_lookup['len_state']==1) & (fips_lookup['len_county']==1),fips_lookup['clean_state']+"00"+fips_lookup['clean_county'],fips_lookup['fips_clean'])
fips_lookup['fips_clean']=np.where(fips_lookup['len_county']==1,fips_lookup['clean_state']+"00"+fips_lookup['clean_county'],fips_lookup['fips_clean'])

fips_lookup['fips_clean']=np.where((fips_lookup['clean_state'].isin(['10','20','30','40','50'])) & (fips_lookup['len_county']==1),fips_lookup['clean_state']+"00"+fips_lookup['clean_county'],fips_lookup['fips_clean'])
fips_lookup['fips_clean']=np.where((fips_lookup['clean_state'].isin(['10','20','30','40','50'])) & (fips_lookup['len_county']==2),fips_lookup['clean_state']+"0"+fips_lookup['clean_county'],fips_lookup['fips_clean'])




In [62]:
# fips_lookup[fips_lookup['clean_state'].isin(['10','20','30','40','50'])]

In [63]:
###merge
manu_m=manu_lm.merge(fips_lookup,left_on='area_fips',right_on='fips_clean',how='inner',indicator=True)

###limit columns
manu_m=manu_m[['msa_name','annual_avg_estabs']].groupby('msa_name',as_index=False).sum()
len(manu_m['msa_name'].unique())

260

#### Federal Spending

In [64]:
##import state name to fips lookup
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/CCDB_updated_vars/state_fips_lookup.xlsx')

##merge on to federal spending data
fed_spend_m=fed_spend.merge(state_lookup,on='state',how='inner')


In [65]:
###limit acs data to one sample
acs_lm=acs[(acs['sample']=='2015-2019, ACS 5-year') & (acs['met2013']!='Not in identifiable area')]
# state_msa_prop_lm=state_msa_prop[(state_msa_prop['met2013']!='Not in identifiable area') & (state_msa_prop['year']!=0) ]

###group acs data by 
state_msa_prop=acs_lm[['met2013','statefip','year']].groupby(['met2013','statefip'],as_index=False).count()

###now get totals across each msa
state_msa_prop_tot=acs_lm.groupby('statefip',as_index=False).count()
state_msa_prop_tot=state_msa_prop_tot[['year','statefip']].rename(columns={'year':'year_tot'})
state_msa_prop=state_msa_prop.merge(state_msa_prop_tot,on='statefip')
state_msa_prop['prop_msa']=state_msa_prop['year']/state_msa_prop['year_tot']

msa_spend=state_msa_prop.merge(fed_spend_m,left_on='statefip',right_on='state')
msa_spend['govt_spend']=msa_spend['prop_msa']*msa_spend['federal_spend']
msa_spend=msa_spend.groupby(['met2013'],as_index=False).agg({'govt_spend':sum})


In [66]:
###get income median
inc=acs_lm[['met2013','inctot']].groupby(['met2013'],as_index=False).median()
inc['msa_name']=inc['met2013']
inc=inc[inc['met2013']!='Not in identifiable area']

#### Gather all data and match to new msa

In [67]:
####clean pop data
pop_race=pop_race.rename(columns={'Geographic Area Name':'msa_name','Estimate!!SEX AND AGE!!Total population':'total_population','Estimate!!RACE!!Total population!!One race!!White':'pop_white'})
pop_race=pop_race[pop_race['pop_white']!='N']
pop_race['prop_white']=pop_race['pop_white'].astype(float)/pop_race['total_population'].astype(float)
pop_race=pop_race[['msa_name','total_population','pop_white']]

####clean crime data
crime_m=crime_m.rename(columns={'msa_clean':'msa_name'})
crime_m=crime_m[['msa_name','crime_per_cap']]

###clean msa spend
msa_spend=msa_spend.rename(columns={'met2013':'msa_name'})
msa_spend=msa_spend[['msa_name','govt_spend']]
msa_spend=msa_spend[msa_spend['msa_name']!='Not in identifiable area']


In [68]:
crime_m[crime_m['msa_name']=='chicago-naperville-elgin, il-in-wi']

,msa_name,crime_per_cap
337,"chicago-naperville-elgin, il-in-wi",0.000473


In [69]:
#### Process Data Merge by MSA

In [70]:
#### population data

#### Process Data Merge by MSA

###process msa name variable to match 2019 lookup
###process lookup var

#lookup=lookup[~(lookup['metarea_19']=='not in identifiable area')]
# new_msas['msa_city']=new_msas['metarea_19'].str.lower().replace('--','-').apply(lambda x : x.split(",")[0])
# new_msas['state']=new_msas['metarea_19'].str.lower().apply(lambda x : x.split(",")[1]).str.strip()


In [71]:
# l=[' metro area',' micro area',' m.d.',' m.s.a','met2013']

# #### create lookup between old and new msa names
# old_msas['msa_city']=old_msas['metarea_07_lookup'].str.lower().apply(lambda x : x.split(",")[0]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-')
# old_msas['state']=old_msas['metarea_07_lookup'].str.lower().apply(lambda x : x.split(",")[1]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-').str.replace('/','-')

# ####make list of cities,states in msa in census data
# old_msas['city_list']=old_msas['msa_city'].apply(lambda x: x.split('-'))
# old_msas['state_list']=old_msas['state'].apply(lambda x: x.split('-'))


# ####make list of cities, states in msa in lookup data
# new_msas['city_list']=new_msas['msa_city'].apply(lambda x: x.split('-'))
# new_msas['state_list']=new_msas['state'].apply(lambda x: x.split('-'))
# new_msas['city_state_list']=new_msas['city_list']+new_msas['state_list']

# new_msas['city_match']=''
# new_msas['state_match']=''

# ###loop through cities in acs lookup table
# for index1, row1 in new_msas.iterrows():

#     for city1 in row1['city_list']:
#         for state1 in row1['state_list']:

#     ###loop through cities in census data 
#             for index2, row2 in old_msas.iterrows():
#                 for city2 in row2['city_list']:
#                     for state2 in row2['state_list']:

#                         if (city1 == city2) and (state1 == state2):
#                             print(True)
#                             city=row2['city_list']
#                             state=row2['state_list']
#                             new_msas.at[index1,'city_match']=city
#                             new_msas.at[index1,'state_match']=state

# ###re-format list vars as string vars
# new_msas['city_match_str']=new_msas['city_match'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))
# new_msas['state_match_str']=new_msas['state_match'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))

# old_msas['city_match_str']=old_msas['city_list'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))
# old_msas['state_match_str']=old_msas['state_list'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))

 


In [72]:
# new_msas['metarea_19']=new_msas['msa_city']+", "+new_msas['state']
# new_msas['metarea_06_cl']=new_msas['city_match_str']+", "+new_msas['state_match_str']
# new_msas=new_msas[['metarea_19','metarea_06_cl']]


# old_msas['metarea_06']=old_msas['metarea_07_lookup']
# old_msas['metarea_06_cl']=old_msas['city_match_str']+", "+old_msas['state_match_str']
# old_msas=old_msas[['metarea_06','metarea_06_cl']]

In [38]:
###limit data and merge

lookup=new_msas.merge(old_msas,on='metarea_06_cl',how='left').merge(manual,on='metarea_19',how='left')
lookup['metarea_06']=np.where((lookup['metarea_06']=='' )|(lookup['metarea_06'].isnull() ),lookup['metarea_07_lookup'],lookup['metarea_06'])
lookup=lookup[['metarea_19','metarea_06']].rename(columns={'metarea_06':'metarea_07_lookup'})

####get dropped msas
dropped0=[x for x in old_msas['metarea_06'].unique() if x not in lookup['metarea_07_lookup'].unique() ]

####group
lookup=lookup.groupby(['metarea_19','metarea_07_lookup'],as_index=False).sum()


In [88]:
lookup=pd.read_excel('/Users/hannahkamen/Downloads/msa_2019_lookup.xlsx')
lookup=lookup[~(lookup['metarea_19']=='not in identifiable area')]
del lookup['metarea_07_lookup']

#### Code for matching by one county

In [93]:

datasets=[pop_race,crime_m,manu_m,msa_spend,inc]
l=[' metro area',' micro area',' m.d.',' m.s.a','met2013']
for df in datasets:

    df['msa_city']=df['msa_name'].str.lower().apply(lambda x : x.split(",")[0]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-')
    df['state']=df['msa_name'].str.lower().apply(lambda x : x.split(",")[1]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-')

    ####make list of cities,states in msa in census data
    df['city_list']=df['msa_city'].apply(lambda x: x.split('-'))
    df['state_list']=df['state'].apply(lambda x: x.split('-'))


    ####make list of cities, states in msa in lookup data
    lookup['msa_city']=lookup['metarea_19'].str.lower().apply(lambda x : x.split(",")[0]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-')
    lookup['state']=lookup['metarea_19'].str.lower().apply(lambda x : x.split(",")[1]).str.replace('|'.join(l), '', regex=True).str.strip().str.replace('--','-')

    
    lookup['city_list']=lookup['msa_city'].apply(lambda x: x.split('-'))
    lookup['state_list']=lookup['state'].apply(lambda x: x.split('-'))
    lookup['city_state_list']=lookup['city_list']+lookup['state_list']

    lookup['city_match']=''
    lookup['state_match']=''
    
    df['city_match']=''
    df['state_match']=''

    ###loop through cities in acs lookup table
    for index1, row1 in lookup.iterrows():
        for city1 in row1['city_list']:
            for state1 in row1['state_list']:

        ###loop through cities in census data 
                for index2, row2 in df.iterrows():
                    for city2 in row2['city_list']:
                        for state2 in row2['state_list']:
                            if (city1 == city2) and (state1 == state2):
                                print(True)
                                
                                city=row1['city_list']
                                state=row1['state_list']
                                
                                print(city)
                                print(state)
                                

                                df.at[index2,'city_match']=city
                                df.at[index2,'state_match']=state
                                    

#                                 row1['city_match'] =row2['city_list']
#                                 row1['state_match'] =row2['state_list']


    ###re-format list vars as string vars
    lookup['city_match_str']=lookup['city_list'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))
    lookup['state_match_str']=lookup['state_list'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))

    df['city_match_str']=df['city_match'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))
    df['state_match_str']=df['state_match'].astype(str).apply(lambda x: x.replace(']','').replace('[','').replace(',','-').replace("'",'').replace("- ",'-'))

 


True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['allentown', 'bethlehem', 'easton']
['pa', 'nj']
True
['anniston', 'oxford', 'jacksonville']
['al']
True
['anniston', 'oxford', 'jacksonville']
['al']
True
['atlanta', 'sandy springs', 'roswell']
['ga']
True
['atlanta', 'sandy springs', 'roswell']
['ga']
True
['atlantic city', 'hammonton']
['nj']
True
['atlantic city', 'hammonton']
['nj']
True
['auburn', 'opelika']
['al']
True
['auburn', 'opelika']
['al']
True
['augusta', 'richmond county']
['ga', 'sc']
True
['augusta', 'richmond county']
['ga', 'sc']
True
['augusta', 'richmond county']
['ga', 'sc']
True
['augusta', 'richmond county']
['ga', 'sc']
True
['austin', 'round rock']
['tx']
True
['austin', 'round rock']
['tx']
True
['baltimore', 'columbia', 'towson']

True
['miami', 'fort lauderdale', 'west palm beach']
['fl']
True
['miami', 'fort lauderdale', 'west palm beach']
['fl']
True
['michigan city', 'la porte']
['in']
True
['michigan city', 'la porte']
['in']
True
['midland']
['tx']
True
['milwaukee', 'waukesha', 'west allis']
['wi']
True
['milwaukee', 'waukesha', 'west allis']
['wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['minneapolis', 'st. paul', 'bloomington']
['mn', 'wi']
True
['monroe']
['mi']
True
['morgantown']
['wv']
True
['muskegon']
['mi']
True
['myrtle beach', 'conway', 'north myrtle beach']
['sc', 'nc']
True
['myrtle beach', 'conway', 'north myrtle beach']
['sc', 'nc']
True
['myrtle beach', 'conway', 'north myrtle beach']
['sc', 'nc']
True
['myrtle beach', 'con

True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['wenatchee']
['wa']
True
['winston', 'salem']
['nc']
True
['winston', 'salem']
['nc']
True
['worcester']
['ma', 'ct']
True
['worcester']
['ma', 'ct']
True
['york', 'hanover']
['pa']
True
['york', 'hanover']
['pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['youngstown', 'warren', 'boardman']
['oh', 'pa']
True
['akron']
['oh']
True
['albany', 'schenectady', 'troy']
['ny']
True
['albany', 'schenectady', 'troy']
['ny']
True
['albany', 'schenectady', 'troy']
['ny']
True
['albuquerque']
['nm']
True
['amarillo']
['tx']
True
['anchorage']
['ak']
True
['ann arbor']
['mi']
True
['asheville']
['nc']
True
['bakersfield']
['ca']
True
['baton rouge']
['la']
True
['bellingham']
['wa']
True

True
['college station', 'bryan']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['dallas', 'fort worth', 'arlington']
['tx']
True
['daphne', 'fairhope', 'foley']
['al']
True
['daphne', 'fairhope', 'foley']
['al']
True
['daphne', 'fairhope', 'foley']
['al']
True
['dayton']
['oh']
True
['dayton']
['oh']
True
['deltona', 'daytona beach', 'ormond beach']
['fl']
True
['deltona', 'daytona beach', 'ormond beach']
['fl']
True
['deltona', 'daytona beach', 'ormond beach']
['fl']
True
['denver', 'aurora', 'lakewood']
['co']
True
['denver', 'aurora', 'lakewood']
['co']
True
['denver', 'aurora', 'lakewood']
['co']
True
['des moines', 'west des moines']
['ia']
True
['des moines', 'west des

True
['new york', 'newark', 'jersey city']
['ny', 'nj', 'pa']
True
['niles', 'benton harbor']
['mi']
True
['niles', 'benton harbor']
['mi']
True
['niles', 'benton harbor']
['mi']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['norwich', 'new london']
['ct']
True
['norwich', 'new london']
['ct']
True
['ocean city']
['nj']
True
['ogden', 'clearfield']
['ut']
True
['ogden', 'clearfield']
['ut']
True
['olympia', 'tumwater']
['wa']
True
['olympia', 'tumwater']
['wa']
True
['olympia', 'tumwater']
['wa']
True
['olympia', 'tumwater']
['wa']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['oshkosh', '

...

True
['bridgeport', 'stamford', 'norwalk']
['ct']
True
['bridgeport', 'stamford', 'norwalk']
['ct']
True
['bridgeport', 'stamford', 'norwalk']
['ct']
True
['brownsville', 'harlingen']
['tx']
True
['brownsville', 'harlingen']
['tx']
True
['buffalo', 'cheektowaga', 'niagara falls']
['ny']
True
['buffalo', 'cheektowaga', 'niagara falls']
['ny']
True
['buffalo', 'cheektowaga', 'niagara falls']
['ny']
True
['burlington']
['nc']
True
['burlington', 'south burlington']
['vt']
True
['burlington', 'south burlington']
['vt']
True
['canton', 'massillon']
['oh']
True
['canton', 'massillon']
['oh']
True
['cape coral', 'fort myers']
['fl']
True
['cape coral', 'fort myers']
['fl']
True
['champaign', 'urbana']
['il']
True
['champaign', 'urbana']
['il']
True
['charleston']
['wv']
True
['charleston', 'north charleston']
['sc']
True
['charleston', 'north charleston']
['sc']
True
['charlotte', 'concord', 'gastonia']
['nc', 'sc']
True
['charlotte', 'concord', 'gastonia']
['nc', 'sc']
True
['charlotte', 'co

True
['niles', 'benton harbor']
['mi']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['north port', 'sarasota', 'bradenton']
['fl']
True
['norwich', 'new london']
['ct']
True
['norwich', 'new london']
['ct']
True
['ocean city']
['nj']
True
['ogden', 'clearfield']
['ut']
True
['ogden', 'clearfield']
['ut']
True
['olympia', 'tumwater']
['wa']
True
['olympia', 'tumwater']
['wa']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['omaha', 'council bluffs']
['ne', 'ia']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['orlando', 'kissimmee', 'sanford']
['fl']
True
['oshkosh', 'neenah']
['wi']
True
['oshkosh', 'neenah']
['wi']
True
['owensboro']
['ky']
True
['oxnard', 'thousand oaks', 'ventura']
['ca']
True
['oxnard', 'thousand oaks', 'ventura']
['ca']
True
['oxnard', 'thousand oaks', '

True
['medford']
['or']
True
['merced']
['ca']
True
['mobile']
['al']
True
['modesto']
['ca']
True
['monroe']
['la']
True
['montgomery']
['al']
True
['muncie']
['in']
True
['ocala']
['fl']
True
['odessa']
['tx']
True
['oklahoma city']
['ok']
True
['pittsburgh']
['pa']
True
['provo', 'orem']
['ut']
True
['provo', 'orem']
['ut']
True
['pueblo']
['co']
True
['punta gorda']
['fl']
True
['racine']
['wi']
True
['reading']
['pa']
True
['redding']
['ca']
True
['reno']
['nv']
True
['roanoke']
['va']
True
['rochester']
['ny']
True
['rockford']
['il']
True
['rocky mount']
['nc']
True
['santa fe']
['nm']
True
['sheboygan']
['wi']
True
['springfield']
['il']
True
['springfield']
['mo']
True
['state college']
['pa']
True
['syracuse']
['ny']
True
['tampa', 'st. petersburg', 'clearwater']
['fl']
True
['tampa', 'st. petersburg', 'clearwater']
['fl']
True
['tampa', 'st. petersburg', 'clearwater']
['fl']
True
['topeka']
['ks']
True
['trenton']
['nj']
True
['tucson']
['az']
True
['tuscaloosa']
['al']
True

True
['lawrence']
['ks']
True
['lebanon']
['pa']
True
['lewiston', 'auburn']
['me']
True
['lewiston', 'auburn']
['me']
True
['little rock', 'north little rock', 'conway']
['ar']
True
['little rock', 'north little rock', 'conway']
['ar']
True
['little rock', 'north little rock', 'conway']
['ar']
True
['los angeles', 'long beach', 'anaheim']
['ca']
True
['los angeles', 'long beach', 'anaheim']
['ca']
True
['los angeles', 'long beach', 'anaheim']
['ca']
True
['louisville/jefferson county']
['ky', 'in']
True
['louisville/jefferson county']
['ky', 'in']
True
['madera']
['ca']
True
['manchester', 'nashua']
['nh']
True
['manchester', 'nashua']
['nh']
True
['mcallen', 'edinburg', 'mission']
['tx']
True
['mcallen', 'edinburg', 'mission']
['tx']
True
['mcallen', 'edinburg', 'mission']
['tx']
True
['memphis']
['tn', 'ms', 'ar']
True
['memphis']
['tn', 'ms', 'ar']
True
['memphis']
['tn', 'ms', 'ar']
True
['miami', 'fort lauderdale', 'west palm beach']
['fl']
True
['miami', 'fort lauderdale', 'west

True
['visalia', 'porterville']
['ca']
True
['visalia', 'porterville']
['ca']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['washington', 'arlington', 'alexandria']
['dc', 'va', 'md', 'wv']
True
['wenatchee']
['wa']
True
['winston', 'salem']
['nc']

In [102]:
master2

,metarea_19,msa_city_x,state_x,city_list_x,state_list_x,city_state_list,city_match_x,state_match_x,city_match_str,state_match_str,...,pop_white,msa_city_y,state_y,city_list_y,state_list_y,city_match_y,state_match_y,msa_name_y,crime_per_cap,_merge
0,"allentown-bethlehem-easton, pa-nj",allentown-bethlehem-easton,pa-nj,"[allentown, bethlehem, easton]","[pa, nj]","[allentown, bethlehem, easton, pa, nj]",,,allentown-bethlehem-easton,pa-nj,...,705095,allentown-bethlehem-easton,pa-nj,"[allentown, bethlehem, easton]","[pa, nj]","[allentown, bethlehem, easton]","[pa, nj]","allentown-bethlehem-easton, pa-nj",0.016024,both
1,"anniston-oxford-jacksonville, al",anniston-oxford-jacksonville,al,"[anniston, oxford, jacksonville]",[al],"[anniston, oxford, jacksonville, al]",,,anniston-oxford-jacksonville,al,...,82323,anniston-oxford,al,"[anniston, oxford]",[al],"[anniston, oxford, jacksonville]",[al],"anniston-oxford-jacksonville, al",0.037028,both
2,"atlanta-sandy springs-roswell, ga",atlanta-sandy springs-roswell,ga,"[atlanta, sandy springs, roswell]",[ga],"[atlanta, sandy springs, roswell, ga]",,,atlanta-sandy springs-roswell,ga,...,3143242,atlanta-sandy springs-alpharetta,ga,"[atlanta, sandy springs, alpharetta]",[ga],"[atlanta, sandy springs, roswell]",[ga],"atlanta-sandy springs-roswell, ga",0.03191,both
3,"atlantic city-hammonton, nj",atlantic city-hammonton,nj,"[atlantic city, hammonton]",[nj],"[atlantic city, hammonton, nj]",,,atlantic city-hammonton,nj,...,168846,atlantic city-hammonton,nj,"[atlantic city, hammonton]",[nj],"[atlantic city, hammonton]",[nj],"atlantic city-hammonton, nj",0.026553,both
4,"auburn-opelika, al",auburn-opelika,al,"[auburn, opelika]",[al],"[auburn, opelika, al]",,,auburn-opelika,al,...,116738,auburn-opelika,al,"[auburn, opelika]",[al],"[auburn, opelika]",[al],"auburn-opelika, al",0.028346,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,"wichita falls, tx",wichita falls,tx,[wichita falls],[tx],"[wichita falls, tx]",,,wichita falls,tx,...,125613,wichita falls,tx,[wichita falls],[tx],[wichita falls],[tx],"wichita falls, tx",0.027994,both
256,"wilmington, nc",wilmington,nc,[wilmington],[nc],"[wilmington, nc]",,,wilmington,nc,...,239181,wilmington,nc,[wilmington],[nc],[wilmington],[nc],"wilmington, nc",0.022373,both
257,"yakima, wa",yakima,wa,[yakima],[wa],"[yakima, wa]",,,yakima,wa,...,189216,yakima,wa,[yakima],[wa],[yakima],[wa],"yakima, wa",0.027697,both
258,"yuba city, ca",yuba city,ca,[yuba city],[ca],"[yuba city, ca]",,,yuba city,ca,...,128517,yuba city,ca,[yuba city],[ca],[yuba city],[ca],"yuba city, ca",0.028302,both


In [120]:
master5.head(50)

,metarea_19,msa_city_x,state_x,city_list_x,state_list_x,city_state_list,city_match_x,state_match_x,city_match_str,state_match_str,...,state_match_y,met2013,inctot,msa_name,msa_city,state,city_list,state_list,city_match,state_match
0,"allentown-bethlehem-easton, pa-nj",allentown-bethlehem-easton,pa-nj,"[allentown, bethlehem, easton]","[pa, nj]","[allentown, bethlehem, easton, pa, nj]",,,allentown-bethlehem-easton,pa-nj,...,"[pa, nj]","Allentown-Bethlehem-Easton, PA-NJ",36503.0,"Allentown-Bethlehem-Easton, PA-NJ",allentown-bethlehem-easton,pa-nj,"[allentown, bethlehem, easton]","[pa, nj]","[allentown, bethlehem, easton]","[pa, nj]"
1,"anniston-oxford-jacksonville, al",anniston-oxford-jacksonville,al,"[anniston, oxford, jacksonville]",[al],"[anniston, oxford, jacksonville, al]",,,anniston-oxford-jacksonville,al,...,[al],"Anniston-Oxford-Jacksonville, AL",24503.0,"Anniston-Oxford-Jacksonville, AL",anniston-oxford-jacksonville,al,"[anniston, oxford, jacksonville]",[al],"[anniston, oxford, jacksonville]",[al]
2,"atlanta-sandy springs-roswell, ga",atlanta-sandy springs-roswell,ga,"[atlanta, sandy springs, roswell]",[ga],"[atlanta, sandy springs, roswell, ga]",,,atlanta-sandy springs-roswell,ga,...,[ga],"Atlanta-Sandy Springs-Roswell, GA",40153.0,"Atlanta-Sandy Springs-Roswell, GA",atlanta-sandy springs-roswell,ga,"[atlanta, sandy springs, roswell]",[ga],"[atlanta, sandy springs, roswell]",[ga]
3,"atlantic city-hammonton, nj",atlantic city-hammonton,nj,"[atlantic city, hammonton]",[nj],"[atlantic city, hammonton, nj]",,,atlantic city-hammonton,nj,...,[nj],"Atlantic City-Hammonton, NJ",33374.0,"Atlantic City-Hammonton, NJ",atlantic city-hammonton,nj,"[atlantic city, hammonton]",[nj],"[atlantic city, hammonton]",[nj]
4,"auburn-opelika, al",auburn-opelika,al,"[auburn, opelika]",[al],"[auburn, opelika, al]",,,auburn-opelika,al,...,[al],"Auburn-Opelika, AL",28000.0,"Auburn-Opelika, AL",auburn-opelika,al,"[auburn, opelika]",[al],"[auburn, opelika]",[al]
5,"augusta-richmond county, ga-sc",augusta-richmond county,ga-sc,"[augusta, richmond county]","[ga, sc]","[augusta, richmond county, ga, sc]",,,augusta-richmond county,ga-sc,...,"[ga, sc]","Augusta-Richmond County, GA-SC",30544.0,"Augusta-Richmond County, GA-SC",augusta-richmond county,ga-sc,"[augusta, richmond county]","[ga, sc]","[augusta, richmond county]","[ga, sc]"
6,"austin-round rock, tx",austin-round rock,tx,"[austin, round rock]",[tx],"[austin, round rock, tx]",,,austin-round rock,tx,...,[tx],"Austin-Round Rock, TX",45000.0,"Austin-Round Rock, TX",austin-round rock,tx,"[austin, round rock]",[tx],"[austin, round rock]",[tx]
7,"baltimore-columbia-towson, md",baltimore-columbia-towson,md,"[baltimore, columbia, towson]",[md],"[baltimore, columbia, towson, md]",,,baltimore-columbia-towson,md,...,[md],"Baltimore-Columbia-Towson, MD",45815.0,"Baltimore-Columbia-Towson, MD",baltimore-columbia-towson,md,"[baltimore, columbia, towson]",[md],"[baltimore, columbia, towson]",[md]
8,"bangor, me",bangor,me,[bangor],[me],"[bangor, me]",,,bangor,me,...,[me],"Bangor, ME",26471.0,"Bangor, ME",bangor,me,[bangor],[me],[bangor],[me]
9,"barnstable town, ma",barnstable town,ma,[barnstable town],[ma],"[barnstable town, ma]",,,barnstable town,ma,...,[ma],"Barnstable Town, MA",41438.0,"Barnstable Town, MA",barnstable town,ma,[barnstable town],[ma],[barnstable town],[ma]


In [103]:
########merge data
master1=lookup.merge(pop_race,on=['city_match_str','state_match_str'],how='left',indicator=True)
dropped1=master1[master1['_merge']=='left_only']['metarea_19'].unique()
master1=master1[master1['_merge']=='both']
del master1['_merge']


crime_m['msa_name']=crime_m['msa_name'].str.replace('raleigh-cary, nc','raleigh, nc')
crime_m['msa_name']=crime_m['msa_name'].str.replace('sacramento–roseville–arden-arcade, ca','sacramento--roseville--arden-arcade, ca')
crime_m['msa_name']=crime_m['msa_name'].str.replace('scranton–wilkes-barre–hazleton, pa','scranton--wilkes-barre--hazleton, pa')
crime_m['msa_name']=crime_m['msa_name'].str.replace('nashville-davidson–murfreesboro–franklin, tn','nashville-davidson--murfreesboro--franklin, tn')
crime_m['msa_name']=crime_m['msa_name'].str.replace('fayetteville-springdale-rogers, ar-mo-mo','fayetteville-springdale-rogers, ar-mo')


master2=master1.merge(crime_m[['msa_name','crime_per_cap']],left_on='metarea_19',right_on='msa_name',how='left',indicator=True)
dropped2=master2[master2['_merge']=='left_only']['metarea_19'].unique()
master2=master2[master2['_merge']=='both']
del master2['_merge']

master3=master2.merge(manu_m,on=['city_match_str','state_match_str'],how='left',indicator=True)
dropped3=master3[master3['_merge']=='left_only']['metarea_19'].unique()
master3=master3[master3['_merge']=='both']
del master3['_merge']

master4=master3.merge(msa_spend,on=['city_match_str','state_match_str'],how='left',indicator=True)
dropped4=master4[master4['_merge']=='left_only']['metarea_19'].unique()
master4=master4[master4['_merge']=='both']
del master4['_merge']

master5=master4.merge(inc,on=['city_match_str','state_match_str'],how='left',indicator=True)
dropped5=master5[master5['_merge']=='left_only']['metarea_19'].unique()
master5=master5[master5['_merge']=='both']
del master5['_merge']
# df_m['msa_2013']=df_m['msa_city_y'].str.strip()+", "+df_m['state_y'].str.strip()                                                       


In [105]:
master_lm=master5[['metarea_19','total_population','pop_white','crime_per_cap','annual_avg_estabs','govt_spend','inctot']]
master_lm['prop_white']=master_lm['pop_white'].astype(float)/master_lm['total_population']
master_lm['crime_per_cap']=master_lm['crime_per_cap'].astype(float)
master_lm=master_lm.rename(columns={'metarea_19':'metarea'})
master_lm['gov_per_cap']=master_lm['govt_spend']/master_lm['total_population']
master_lm=master_lm[(master_lm['govt_spend']!=0)&~(master_lm['govt_spend'].isnull())]
# master_lm=master_lm[~(master_lm['metarea_07_lookup']=='')]


In [110]:
lookup=pd.read_excel('/Users/hannahkamen/Downloads/msa_2019_lookup.xlsx')

,metarea_19,msa_city,state,city_list,state_list,city_state_list,city_match,state_match,city_match_str,state_match_str
1,"allentown-bethlehem-easton, pa-nj",allentown-bethlehem-easton,pa-nj,"[allentown, bethlehem, easton]","[pa, nj]","[allentown, bethlehem, easton, pa, nj]",,,allentown-bethlehem-easton,pa-nj
2,"anniston-oxford-jacksonville, al",anniston-oxford-jacksonville,al,"[anniston, oxford, jacksonville]",[al],"[anniston, oxford, jacksonville, al]",,,anniston-oxford-jacksonville,al
3,"atlanta-sandy springs-roswell, ga",atlanta-sandy springs-roswell,ga,"[atlanta, sandy springs, roswell]",[ga],"[atlanta, sandy springs, roswell, ga]",,,atlanta-sandy springs-roswell,ga
4,"atlantic city-hammonton, nj",atlantic city-hammonton,nj,"[atlantic city, hammonton]",[nj],"[atlantic city, hammonton, nj]",,,atlantic city-hammonton,nj
5,"auburn-opelika, al",auburn-opelika,al,"[auburn, opelika]",[al],"[auburn, opelika, al]",,,auburn-opelika,al
...,...,...,...,...,...,...,...,...,...,...
256,"wichita falls, tx",wichita falls,tx,[wichita falls],[tx],"[wichita falls, tx]",,,wichita falls,tx
257,"wilmington, nc",wilmington,nc,[wilmington],[nc],"[wilmington, nc]",,,wilmington,nc
258,"yakima, wa",yakima,wa,[yakima],[wa],"[yakima, wa]",,,yakima,wa
259,"yuba city, ca",yuba city,ca,[yuba city],[ca],"[yuba city, ca]",,,yuba city,ca


In [109]:
master_lm.merge()

,metarea,total_population,pop_white,crime_per_cap,annual_avg_estabs,govt_spend,inctot,prop_white,gov_per_cap
0,"allentown-bethlehem-easton, pa-nj",844052,705095,0.016024,4727,2.499027e+06,36503.0,0.835369,2.960750
1,"anniston-oxford-jacksonville, al",113605,82323,0.037028,703,4.445238e+05,24503.0,0.724642,3.912889
2,"atlanta-sandy springs-roswell, ga",6018744,3143242,0.031910,24864,1.499196e+07,40153.0,0.522242,2.490878
3,"atlantic city-hammonton, nj",263670,168846,0.026553,592,5.298036e+05,33374.0,0.640369,2.009343
4,"auburn-opelika, al",164542,116738,0.028346,727,4.793201e+05,28000.0,0.709472,2.913057
...,...,...,...,...,...,...,...,...,...
249,"wichita falls, tx",151254,125613,0.027994,689,3.757722e+05,26634.0,0.830477,2.484379
250,"wilmington, nc",297533,239181,0.022373,1235,7.945277e+05,32580.0,0.803881,2.670385
251,"yakima, wa",250873,189216,0.027697,1335,6.177337e+05,31288.0,0.754230,2.462336
252,"yuba city, ca",175639,128517,0.028302,529,5.363664e+05,32373.0,0.731711,3.053800


In [106]:
print(len(dropped1))
print(len(dropped2))
print(len(dropped3))
print(len(dropped4))
print(len(dropped5))

0
6
0
0
0


In [108]:
dropped2

array(['blacksburg-christiansburg-radford, va',
       'fayetteville-springdale-rogers, ar-mo',
       'hickory-lenoir-morganton, nc', 'lawrence, ks',
       'winston-salem, nc', 'youngstown-warren-boardman, oh-pa'],
      dtype=object)

### Diagnostics for albuoy data merge

In [111]:
###import puma lookup and albuoy data

###import met2013 to PUMA2000 lookup
puma_lkup=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/msa2013_puma2000.dta')



###import albuoy data
albuoy=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/albouy_all.dta')

###limit prop by state data
state_msa_prop_lm=state_msa_prop[state_msa_prop['prop_msa']>0]

In [116]:
len(puma_lkup['msaname_crosswalk'].unique())

258

In [114]:
puma_lkup[puma_lkup['msaname_crosswalk'].str.contains('Denver')]

,met2013,msaname_crosswalk,statefip,statename,pctmsapop,pctpumapop,PumaID,dup
22,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,0.000229,0.004421,800802.0,0.0
31,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,0.006332,0.076270,800300.0,0.0
92,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,0.666379,14.383765,801000.0,2.0
95,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,0.911814,14.886397,800400.0,0.0
214,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,3.134682,67.110356,800804.0,0.0
244,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,4.197088,85.116223,800801.0,0.0
529,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,4.630190,100.000000,800814.0,0.0
700,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,4.763530,100.000000,800809.0,0.0
760,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,4.629181,100.000000,800818.0,0.0
761,19740,"Denver-Aurora-Lakewood, CO",8,Colorado,5.034523,100.000000,800817.0,0.0


In [46]:
#fix new hampshire(33), Vermont(50), West Virginia(54), Wyoming(56), North Dakota(38), South Dakota(46), Montana(30)

####make virtual msas in albuoy data
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, NH']), 996,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, VT','Burlington, VT']), 997,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, WV','Charleston, WV']), 998,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, WY','Casper, WY','Cheyenne, WY']), 999,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, ND']), 1000,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Rapid City, SD','Non-metro, SD','Sioux Falls, SD']), 1001,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, MT','Great Falls, MT','Billings, MT','Missoula, MT']), 1002,albuoy['PumaID'])

####group by new virtual msas, take mean

albuoy=albuoy.groupby(['PumaID','statefip'],as_index=False).mean()
####add rows to lookup
puma_lkup=puma_lkup[['met2013','msaname_crosswalk','statefip','statename','pctmsapop','PumaID']]
puma_lkup.loc[len(puma_lkup.index)] = [996, 'NH Virtual MSA, NH', 33,'New Hampshire', 100, 996] 
puma_lkup.loc[len(puma_lkup.index)] = [997, 'VT Virtual MSA, VT', 50,'Vermont', 100, 997] 
puma_lkup.loc[len(puma_lkup.index)] = [998, 'WV Virtual MSA, WV', 54,'West Virginia', 100, 998] 
puma_lkup.loc[len(puma_lkup.index)] = [999, 'WY Virtual MSA, WY', 56,'Wyoming', 100, 999] 
puma_lkup.loc[len(puma_lkup.index)] = [1000, 'ND Virtual MSA, ND', 38,'North Dakota', 100, 1000] 
puma_lkup.loc[len(puma_lkup.index)] = [1001, 'SD Virtual MSA, SD', 46,'South Dakota', 100, 1001] 
puma_lkup.loc[len(puma_lkup.index)] = [1002, 'MT Virtual MSA, MT', 30,'Montana', 100, 1002] 

###adjust lookup
puma_lkup['msaname_crosswalk_l']=puma_lkup['msaname_crosswalk'].str.lower()



In [47]:
####get percentage of each msa in each state

###limit acs data to one sample
# acs_lm=acs[(acs['sample']=='2015-2019, ACS 5-year') & (acs['met2013']!='Not in identifiable area')]


acs['met2013']=np.where(acs['met2013'].isin(['Manchester-Nashua, NH']), 'NH Virtual MSA, NH',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='New Hampshire') ), 'NH Virtual MSA, NH',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Burlington-South Burlington, VT','Burlington, VT']), 'VT Virtual MSA, VT',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='Vermont') ), 'VT Virtual MSA, VT',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Charleston, WV','Morgantown, WV','Parkersburg-Vienna, WV']), 'WV Virtual MSA, WV',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='West Virginia') ), 'WV Virtual MSA, WV',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Non-metro, WY','Casper, WY','Cheyenne, WY']), 'WY Virtual MSA, WY',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='Wyoming') ),'WY Virtual MSA, WY',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Bismarck, ND']), 'ND Virtual MSA, ND',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='North Dakota') ), 'ND Virtual MSA, ND',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Rapid City, SD','Non-metro, SD','Sioux Falls, SD']), 'SD Virtual MSA, SD',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='South Dakota') ), 'SD Virtual MSA, SD',acs['met2013'])

acs['met2013']=np.where(acs['met2013'].isin(['Non-metro, MT','Great Falls, MT','Billings, MT','Missoula, MT']), 'MT Virtual MSA, MT',acs['met2013'])
acs['met2013']=np.where((acs['met2013'].isin(['Not in identifiable area']) & (acs['statefip']=='Montana') ), 'MT Virtual MSA, MT',acs['met2013'])


###group acs data by 
prop_msa_state=acs[['met2013','statefip','year']].groupby(['met2013','statefip'],as_index=False).count()





###now get totals across each msa
prop_msa_state_tot=acs.groupby('met2013',as_index=False).count()

prop_msa_state_tot=prop_msa_state_tot[['year','met2013']].rename(columns={'year':'year_tot'})
prop_msa_state=prop_msa_state.merge(prop_msa_state_tot,on='met2013')
prop_msa_state['prop_state_in_msa']=prop_msa_state['year']/prop_msa_state['year_tot']



In [48]:
#fix new hampshire(33), Vermont(50), West Virginia(54), Wyoming(56), North Dakota(38), South Dakota(46), Montana(30)


prop_msa_state[prop_msa_state['met2013'].str.contains('|'.join([", NH",', VT',', WV',', WY',', ND',', SD',', MT']))]['met2013'].unique()




array(['MT Virtual MSA, MT', 'ND Virtual MSA, ND', 'NH Virtual MSA, NH',
       'SD Virtual MSA, SD', 'VT Virtual MSA, VT', 'WV Virtual MSA, WV',
       'WY Virtual MSA, WY'], dtype=object)

In [49]:
prop_msa_state_lm=prop_msa_state[prop_msa_state['prop_state_in_msa']>0]
prop_msa_state_lm['msaname_crosswalk_l']=prop_msa_state_lm['met2013'].str.lower()
prop_msa_state_lm=prop_msa_state_lm.rename(columns={'statefip':'statename'})

In [50]:
###merge albuoy data with lookup
albuoy_updated=albuoy.merge(puma_lkup,on=['PumaID','statefip'])
albuoy_updated['propmsapop']=albuoy_updated['pctmsapop']/100

####multiply all columns in albuoy by msa pct share

for col in albuoy_updated.columns:
    if col not in ['statefip', 'PumaID', 'msa', 'msaname','BootstrapMSA', 'BootstrapState', 'met2013', 'msaname_crosswalk',
       'statename', 'pctmsapop', 'pctpumapop', 'dup', 'msaname_crosswalk_l',
       'propmsapop']:
        try:
            albuoy_updated['%s_adj'%col]=albuoy_updated['%s'%col]*albuoy_updated['propmsapop']
        except:
            pass
name_vars=['msaname_crosswalk_l','met2013','statefip','statename']
num_vars=[x for x in albuoy_updated.columns if 'adj' in x ]
albuoy_updated_lm=albuoy_updated[name_vars + num_vars]
albuoy_gr=albuoy_updated_lm.groupby(name_vars,as_index=False).sum()
print(len(albuoy_gr))
###merge on proportion in state
albuoy_gr=albuoy_gr.merge(prop_msa_state_lm[['msaname_crosswalk_l','statename','prop_state_in_msa']],on=['msaname_crosswalk_l','statename'],how='inner')
print(len(albuoy_gr))
########now adjust for percentage of msa in each state for multistatemsas

for col in albuoy_gr.columns:
    if col not in ['msaname_crosswalk_l','met2013','statefip','statename','prop_state_in_msa']:
        try:
            albuoy_gr['%s_2'%col]=albuoy_gr['%s'%col]*albuoy_gr['prop_state_in_msa']
        except:
            pass
name_vars2=['msaname_crosswalk_l','met2013','statefip','statename']
num_vars2=[x for x in albuoy_gr.columns if 'adj_2' in x ]
albuoy_gr2=albuoy_gr[name_vars2 + num_vars2]
albuoy_gr2=albuoy_gr2.groupby(['msaname_crosswalk_l','met2013'],as_index=False).sum()
albuoy_gr2=albuoy_gr2.rename(columns={'msaname_crosswalk_l':'metarea'})

##rename columns

albuoy_gr2.columns=[x.replace('_adj_2','') for x in albuoy_gr2.columns]


297
285


In [51]:
albuoy_gr2[albuoy_gr2['metarea'].str.contains('bend')]

,metarea,met2013,statefip,QOL_fulladj_2,QOL_doubleadj_2,qol_popadj_2,Wage_orig,Wage,Price,QOL_orig,...,DeltaA1F130YearTempAvg9,DeltaA1F130YearRelHum10,DeltaA1F130YearDewPt10,DeltaA1F130YearTempAvg10,DeltaA1F130YearRelHum11,DeltaA1F130YearDewPt11,DeltaA1F130YearTempAvg11,DeltaA1F130YearRelHum12,DeltaA1F130YearDewPt12,DeltaA1F130YearTempAvg12
20,"bend-redmond, or",13460,41,0.08052,0.108351,0.027831,-0.091114,-0.09164,0.124912,0.084294,...,12.025361,-2.311855,7.516562,8.872669,0.795189,7.855449,7.780591,-2.041537,6.845558,7.594061


In [ ]:
len(albuoy_gr2)

In [ ]:
len(albuoy_gr2.groupby('metarea').sum())

In [ ]:
albuoy_gr2.to_stata('/Users/hannahkamen/Documents/population-migration2/dta/albuoy_all_msa_level.dta')

In [ ]:
albuoy_gr[albuoy_gr['msaname_crosswalk_l'].str.contains('new york')]['DeltaA1F130YearTempAvg9_adj']

In [ ]:
albuoy_gr2[albuoy_gr2['metarea'].str.contains('new york')]['DeltaA1F130YearTempAvg9_adj_2']

In [ ]:
albuoy_gr2[albuoy_gr2['metarea'].str.contains('seattle')]

In [ ]:
master_lm['metarea']

In [ ]:
master_lm[master_lm['metarea'].str.contains('nh')]

In [ ]:
list(albuoy_gr)

In [ ]:
test=master_lm.merge(albuoy_gr2,on='metarea',how='left',indicator=True)

In [ ]:
test[test['_merge']!='both']

In [ ]:
master_lm.to_stata('/Users/hannahkamen/Documents/population-migration2/dta/new_ccdb.dta')

In [ ]:
old_ccdb_lm=old_ccdb[['metarea','state','proptax']]
old_ccdb_lm=old_ccdb_lm.rename(columns={'metarea':'metareanum'})

In [ ]:
old_ccdb_lm.to_stata('/Users/hannahkamen/Documents/population-migration2/dta/old_ccdb_lm.dta')